In [1]:
from models.mine.MSTGAT import MSTGAT
from models.GDN import GDN
from util.params import Params
from util.env import set_param
import torch
from torch_geometric.utils import dense_to_sparse
import tqdm
from datasets.TimeDataset import TimeDataset
from util.consts import Tasks,Datasets
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

from util.preprocess import findSensorActuator

param=Params()
param.learning_rate=.001
param.batch=32
param.epoch=60
param.embedding_dimension=128
param.topk=15
kernel_size=16
gamma1=.5
gamma2=.8
param.task=Tasks.next_sensors
param.dataset=Datasets.dummy
set_param(param)


In [19]:
model=GDN(node_num=30)

In [20]:
cp=model.embedding.weight.detach().clone()

In [23]:
x=torch.rand((20,30,param.window_length),requires_grad=False)

In [24]:
opt=torch.optim.Adam(lr=.1,params=model.parameters())

In [25]:
opt.zero_grad()
loss=model(x).mean()-1
loss.backward()
opt.step()

In [27]:
(cp-model.embedding.weight).abs().sum()

tensor(383.9877, grad_fn=<SumBackward0>)

In [54]:
from torch_geometric.nn.models import GAT

gat=GAT(in_channels=10,hidden_channels=30,num_layers=1,out_channels=10,v2=True)

In [55]:
from torch_geometric.utils import dense_to_sparse

index,w=dense_to_sparse(torch.randint(low=0,high=2,size=(20,20)))

In [ ]:
node_num=20
batch_num=40
embedding=torch.nn.Embedding(node_num,60)
embedding.weight

In [64]:
all_embeddings = embedding(torch.arange(node_num))

weights_arr = all_embeddings.detach().clone()
all_embeddings = all_embeddings.repeat(batch_num, 1)

weights = weights_arr.view(node_num, -1)

cos_ji_mat = torch.matmul(weights, weights.T)
normed_mat = torch.matmul(
    weights.norm(dim=-1).view(-1, 1), weights.norm(dim=-1).view(1, -1)
)
cos_ji_mat = cos_ji_mat / normed_mat

dim = weights.shape[-1]
topk_num = 15

topk_indices_ji = torch.topk(cos_ji_mat, topk_num, dim=-1)[1]

learned_graph = topk_indices_ji

In [63]:
learned_graph.shape

torch.Size([20, 15])

In [56]:
x=torch.rand(40,20,10)
gat(x=x,edge_index=index,edge_weight=w)

AssertionError: 

In [2]:
def preapareDF_modal( dataset):
    _train_original = pd.read_csv(
        f"./data/{dataset}/train.csv",
        sep=",",
        index_col=0,
    )
    _test_original = pd.read_csv(
        f"./data/{dataset}/test.csv",
        sep=",",
        index_col=0,
    )

    #     columns = [
    #     col for col in _train_original if col.strip() not in ["datetime", "Timestamp"]
    # ]
    # _train_original=_train_original[columns]
    # _test_original=_test_original[columns]

    for i, col in enumerate(_train_original.columns):
        assert _train_original.columns[i] == _test_original.columns[i]
    stripped_columns = [col.split("_")[0] for col in _train_original.columns]
    adj = torch.zeros(len(stripped_columns), len(stripped_columns))
    for i in range(len(stripped_columns)):
        for j in range(len(stripped_columns)):
            if stripped_columns[i] == stripped_columns[j]:
                adj[i][j] = 1
    return _train_original, _test_original, adj

In [3]:
def prepareDF( scale, dataset):
    _train_original = pd.read_csv(
        f"./data/{dataset}/train.csv", sep=",", index_col=0
    )
    _test_original = pd.read_csv(
        f"./data/{dataset}/test.csv", sep=",", index_col=0
    )
    _columns = findSensorActuator(_train_original)
    sensors, actuators, consts = _columns
    print("#####################################")
    print("sensors count: ", len(sensors))
    print("actuators count: ", len(actuators))
    print("consts count: ", len(consts))
    print("consts: ", consts)
    print("#####################################")
    print(sensors)
    if scale:
        scaler = MinMaxScaler()
        # Initialize the MinMaxScaler
        

        # Fit the scaler on the first dataset
        scaler.fit(_train_original[sensors])

        # Transform both datasets using the same scaler
        _train_original[sensors] = scaler.transform(_train_original[sensors])

        _test_original[sensors] = scaler.transform(_test_original[sensors])
    # if "attack" in _train_original.columns:
    #     _train_original = _train_original.drop(columns=["attack"])
    return _train_original, _test_original, _columns

### Load dataset

In [4]:
train, test, adj=preapareDF_modal(param.dataset.value)
sensors=train.columns
actuators=[]
train_dataset=TimeDataset(                train,
                sensor_list=sensors,
                actuator_list=actuators,
                mode="train",)


### Create dataLoader

In [5]:
train_dataloader=torch.utils.data.DataLoader(dataset=train_dataset,batch_size=param.batch)

### Create model

In [6]:
node_num=len(sensors)
m=MSTGAT(node_num=node_num,adj=adj.cuda(),gamma1=gamma1,gamma2=gamma2,kernel_size=kernel_size)

### TRAIN

In [8]:
opt= torch.optim.Adam(m.parameters(),lr=param.learning_rate)
m=m.train().cuda()
total_samples = train_dataloader.dataset.__len__()
avg_loss=0
for epoch in range(param.epoch):
    t=tqdm.tqdm(
            train_dataloader,
            desc="epoc {} / {}".format((epoch + 1), param.epoch),
            postfix="{loss} {val_loss}",
            position=0,
        )
    i = len(train_dataloader)
    acu_loss=0
    for b, (windowed_x, y, next_label) in enumerate(t):
        i-=1
        opt.zero_grad(set_to_none=True)
        loss=m.loss(windowed_x.cuda(), y.cuda())
        loss.backward()
        t.set_postfix({"loss": (loss.item() / windowed_x.shape[0]), "val_loss": avg_loss})
        opt.step()
        acu_loss+=loss.detach().clone().item()
        if i == 0:
            avg_loss = acu_loss / total_samples

epoc 1 / 60:   0%|          | 0/274 [00:00<?, ?it/s, {loss} {val_loss}]

epoc 1 / 60:   0%|          | 0/274 [00:02<?, ?it/s, {loss} {val_loss}]


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
